<h1 align="center">
Quantum Computer Music
</h1>
<h4 align="right">
Akeel Ather Medina <br>
Habib University <br>
CS/PHY 314/300 <br>
Quantum Computing <br>
Fall 2021 <br>
</h4>

<h2 align="center">
Introduction
</h2>

Since the earliest use of computers, algorithms have been created and used in order to create musical compositions. The kind of algorithm this paper will be focusing on is sequencing rules. A computer is programmed with a certain set of rules that generate a sequence of notes, and they can be in the form of Graphs, Finite State Automata, Markov Chains, etc.

## Random Walk

A random walk can be defined as a path that consists of a succession of random steps over some space. 

<figure align = "center">
  <img src="images/Picture1.png" alt="Random Walk">
  <figcaption>Figure #1</figcaption>
  <figcaption align="center">Source: Quantum Computer Music: Foundations and Initial Experiments (Miranda, Basak)</figcaption>
</figure>

Figure 1 depicts a graph of a connected set of notes. A random walk over this graph would results in a sequence of notes that can be considered a musical composition. This is a simple example of algorithmic music generation.

## Markov Chains

A Markov Chain is a stochastic model that describes a sequence of possible events where the probability of each event depends only on the previous event. The terms 'Markov Chain' and 'Random Walk' can be used interchangeably.

<figure align = "center">
  <img src="images/Picture2.png" alt="Markov Chain">
  <figcaption>Figure #2</figcaption>
  <figcaption align="center">Source: Quantum Computer Music: Foundations and Initial Experiments (Miranda, Basak)</figcaption>
</figure>

Figure 2 depicts the Markov Chain representation of the random walk shown in Figure 1.

## Quantum Random Walk

Quantum Randomness is equivalent to real randomness. Thus, a Quantum Random Walk is truly random. To implement a Quantum Random Walk we can use the idea of a Quantum Die. This die will depend on our number of qubits and the order of For a simple one dimensional random walk such as depicted by Figure 1, we can represent our Quantum Die as such:

<figure align = "center">
  <img src="images/Picture4.png" alt="One-Dimensional Quantum Die">
  <figcaption>Figure #3</figcaption>
  <figcaption align="center">Source: Quantum Computer Music: Foundations and Initial Experiments (Miranda, Basak)</figcaption>
</figure>

<h2 align="center">
Implementation
</h2>

In [1]:
from qiskit import QuantumCircuit,QuantumRegister,ClassicalRegister,execute,Aer
shot_count = 64

When implementing a quantum walk, we want to 'save' the value we receive at the end of each walk. We need each successive circuit to be influenced by this previous value, and this is the necessary circuit that does so. Since a Quantum Circuit starts in the state of $|0⟩$, we add an X gate to every qubit that was previously had a measured value of $|1⟩$.

In [2]:
def updateCircuit(xbar, bits, cbits=0):

    if cbits == 0:
        qc1 = QuantumCircuit(bits)
    else:
        qc1 = QuantumCircuit(bits, cbits)

    qc1.barrier()

    for i in range(len(xbar)):
        if xbar[i] == str(1):
            qc1.x(i)
    
    qc1.barrier()

    return qc1

<h2 align="center">
Quantum Walk over a Tetrahedron
</h2>

## Creating Sequencing Rules for the Random Walk Corresponding to Vertices on a Tetrahedron 

A tetrahedron is defined as a triangular pyramid composed of 4 faces and 4 vertex corners. A quantum Walk over these 4 vertices can be represented by 2 qubits. The corresponding notes to these 4 vertices is derived from the Tetratonic C Scale. An Image describing a walk over a tetrahedron is such:

<figure align = "center">
  <img src="images/Picture7.png" alt="Random Walk">
  <figcaption>Figure #4</figcaption>
</figure>

In [3]:
vertices = {}

vertices['00'] = 'C'
vertices['01'] = 'Eb'
vertices['10'] = 'G'
vertices['11'] = 'A'

rythym = {}

rythym['00'] = 1.0
rythym['01'] = 0.8
rythym['10'] = 0.6
rythym['11'] = 0.4

## Quantum Walk Circuit over a Tetrahedron

The Quantum Walk corresponding to Figure 4 can be implemented as a Quantum circuit with 2 qubits corresponding to each of the 4 vertices and 2 for dice qubits. The dice qubits have 4 possible options:

1. The first qubit is flipped.
2. The second qubit is flipped.
3. Both qubits are flipped.
4. No qubit is flipped.

To implement this, we first put our dice qubits in a superposition, this enables the 'random' in the 'random walk'. We use controlled-X gates to conditionally flip each, both, or none of the qubits.

In [4]:
def tetrahedron_measure_dice(input):

    q_dice = QuantumCircuit(4,2)

    q_inp = updateCircuit(input, 4)

    q_dice += q_inp

    q_dice.h(2)
    q_dice.h(3)

    q_dice.cx(2, 0)
    q_dice.cx(3, 1)
    q_dice.barrier()

    return q_dice

The circuit will look like this with an input state of $|01⟩$:

<figure align = "center">
  <img src="images/Picture5.png" alt="Random Walk">
  <figcaption>Figure #5</figcaption>
  <figcaption align="center">Source: Quantum Computer Music: Foundations and Initial Experiments (Miranda, Basak)</figcaption>
</figure>

## Implementing the Simple Quantum Walk over a Tetrahedron

Starting off with the vertex '00', we do a Quantum Random Walk 10 times, passing the current vertex as an argument into our circuit implementation function. At the end of these iterations we convert the vertices to their respective notes and rythyms, as represnted in the dictionaries given above.

In [5]:
def tetrahedron_quantum_walk():

    start = "00"
    vertex_lst = []
    note_lst = []

    for i in range(10):

        temp = tetrahedron_measure_dice(start)

        temp.measure([0, 1],[0, 1])

        job = execute(temp,Aer.get_backend('qasm_simulator'),shots=shot_count)
        counts = list(job.result().get_counts(temp).items())

        counts.sort(key=lambda x: x[1])

        start = counts[-1][0]
        vertex_lst.append(start)

        # display(temp.draw())

    for i in vertex_lst:
        note_lst.append(vertices[i])

    print(note_lst)

    start = "00"
    vertex_lst = []
    rythym_lst = []

    for i in range(5):

        temp = tetrahedron_measure_dice(start)

        temp.measure([0, 1],[0, 1])

        job = execute(temp,Aer.get_backend('qasm_simulator'),shots=shot_count)
        counts = list(job.result().get_counts(temp).items())

        counts.sort(key=lambda x: x[1])

        start = counts[-1][0]
        vertex_lst.append(start)

    for i in vertex_lst:
        rythym_lst.append(rythym[i])

    print(rythym_lst)

    return note_lst, rythym_lst

notes, rythyms = tetrahedron_quantum_walk()  

C:\Users\akeel\AppData\Local\Temp/ipykernel_5328/786390605.py:7: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  q_dice += q_inp
C:\Users\akeel\AppData\Local\Programs\Python\Python39\lib\site-packages\qiskit\circuit\quantumcircuit.py:942: DeprecationWarning: The QuantumCircuit.extend() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  return self.extend(rhs)


['A', 'Eb', 'A', 'A', 'C', 'A', 'C', 'A', 'G', 'Eb']
[1.0, 0.4, 0.4, 0.4, 0.6]


## Sonic Pi code created Using Quantum Walk over a Tetrahedron

We save the code usable with the application, SonicPi, into text files. The sleep function is added in place for the rhythym, and some sustain and bass is also added.

In [6]:
sonic_note_lst = []
for i in zip(notes, rythyms):
    sonic_note_lst.append("""play :{},sustain:({}),sustain_level:({})\nsample :bass_hit_c\nsleep({})""".format(i[0], i[1], i[1], i[1]))
sonic_note_lst = '\n'.join(sonic_note_lst)

a = open("SonicPiCode/tetrahedron_notefile.txt", "w")
a.write(sonic_note_lst)
a.close()

<h2 align="center">
Quantum Walk over a Cube
</h2>

## Creating Sequencing Rules for the Random Walk Corresponding to Vertices on a Cube 

A cube is composed of 6 faces and 8 vertex corners. A quantum Walk over these 8 vertices can be represented by 3 qubits. The corresponding notes, chords and rhythyms to these 6 vertices is derived from the Persian C Octatonic Scale. An Image describing a walk over a cube is such:

<figure align = "center">
  <img src="images/Picture3.png" alt="Random Walk">
  <figcaption>Figure #6</figcaption>
  <figcaption align="center">Source: Quantum Computer Music: Foundations and Initial Experiments (Miranda, Basak)</figcaption>
</figure>


In [7]:
vertices = {}

vertices['000'] = 'C'
vertices['001'] = 'Db'
vertices['010'] = 'F'
vertices['011'] = 'Gb'
vertices['100'] = 'E'
vertices['101'] = 'B'
vertices['110'] = 'Ab'
vertices['111'] = 'C'

chords = {}

chords['000'] = 'C4 major7'
chords['001'] = 'D4 minor7'
chords['010'] = 'E4 minor7'
chords['011'] = 'F4 major7'
chords['100'] = 'G4 major7'
chords['101'] = 'A4 minor7'
chords['110'] = 'B4 diminished7'
chords['111'] = 'C4 major7'

rythym = {}

rythym['000'] = 1.0
rythym['001'] = 0.9
rythym['010'] = 0.8
rythym['011'] = 0.7
rythym['100'] = 0.6
rythym['101'] = 0.5
rythym['110'] = 0.4
rythym['111'] = 0.3


## Quantum Walk Circuit over a Cube

A 3-qubit Quantum Walk can be implemented as a Quantum circuit with 3 qubits for measurement and 2 for dice qubits. The dice qubits have 4 possible options:

1. The first qubit is flipped.
2. The second qubit is flipped.
3. The third qubit is flipped.
4. No qubit is flipped.

To implement this<sup>[1](#f1)</sup>, we first put our dice qubits in a superposition, this enables the 'random' in the 'random walk'. We use controlled-X gates to conditionally flip each of the qubits, However, we must also ensure that 2 or more bits are not flipped at the same time. This order or gates ensured these conditions and allows us to walk over the defined vertices of our cube. 

<b id="f1">1</b> The circuit implementation has been taken from "Quantum Computer Music: Foundations and Initial Experiments (Miranda, Basak)".

In [8]:
shot_count = 64
def cube_measure_dice(input):

    q_dice = QuantumCircuit(5,3)

    q_inp = updateCircuit(input, 5)

    q_dice += q_inp

    q_dice.h(3)
    q_dice.h(4)

    q_dice.cx(4, 0)
    q_dice.x(4)
    q_dice.cx(4, 1)
    q_dice.cx(3, 2)
    q_dice.mcx([4, 3], 1)
    q_dice.x(4)
    q_dice.mcx([4, 3], 0)
    q_dice.x(4)
    q_dice.mcx([4, 3], 2)
    q_dice.barrier()

    return q_dice

The circuit will look like this with an input state of $|000⟩$:

<figure align = "center">
  <img src="images/Picture8.png" alt="Random Walk">
  <figcaption>Figure #7</figcaption>
  <figcaption align="center">Source: Quantum Computer Music: Foundations and Initial Experiments (Miranda, Basak)</figcaption>
</figure>

## Implementing the Simple Quantum Walk over a Cube

Starting off with the vertex '000', we do a Quantum Random Walk 10 times, passing the current vertex as an argument into our circuit implementation function. At the end of these iterations we convert the vertices to their respective notes, chords, and rythyms, as represnted in the dictionaries given above.

In [9]:
def cube_quantum_walk():

    start = "000"
    vertex_lst = []
    note_lst = []

    for i in range(10):

        temp = cube_measure_dice(start)

        temp.measure([0, 1, 2],[0, 1, 2])

        job = execute(temp,Aer.get_backend('qasm_simulator'),shots=shot_count)
        counts = list(job.result().get_counts(temp).items())

        counts.sort(key=lambda x: x[1])

        start = counts[-1][0]
        vertex_lst.append(start)

    for i in vertex_lst:
        note_lst.append(vertices[i])

    print(note_lst)

    start = "000"
    vertex_lst = []
    chord_lst = []

    for i in range(10):

        temp = cube_measure_dice(start)

        temp.measure([0, 1, 2],[0, 1, 2])

        job = execute(temp,Aer.get_backend('qasm_simulator'),shots=shot_count)
        counts = list(job.result().get_counts(temp).items())

        counts.sort(key=lambda x: x[1])

        start = counts[-1][0]
        vertex_lst.append(start)

    for i in vertex_lst:
        chord_lst.append(chords[i])

    print(chord_lst)

    start = "000"
    vertex_lst = []
    rythym_lst = []

    for i in range(10):

        temp = cube_measure_dice(start)

        temp.measure([0, 1, 2],[0, 1, 2])

        job = execute(temp,Aer.get_backend('qasm_simulator'),shots=shot_count)
        counts = list(job.result().get_counts(temp).items())

        counts.sort(key=lambda x: x[1])

        start = counts[-1][0]
        vertex_lst.append(start)

    for i in vertex_lst:
        rythym_lst.append(rythym[i])

    print(rythym_lst)

    return note_lst, chord_lst, rythym_lst

notes, chords, rythyms = cube_quantum_walk()  

['F', 'F', 'Gb', 'Ab', 'Gb', 'E', 'Db', 'C', 'E', 'C']
['C4 major7', 'E4 minor7', 'C4 major7', 'C4 major7', 'D4 minor7', 'G4 major7', 'D4 minor7', 'C4 major7', 'E4 minor7', 'B4 diminished7']
[0.6, 0.9, 1.0, 0.9, 0.4, 0.8, 1.0, 1.0, 0.9, 0.5]


## Sonic Pi code created Using Quantum Walk over a Cube

We save the code usable with the application, SonicPi, into text files. The sleep function is added in place for the rhythym, and some sustain and bass is also added. The dictionary that maintains the chord list for C major is also used, using the same rhythym values as for the note file.

In [10]:
sonic_note_lst = []
for i in zip(notes, rythyms):
    sonic_note_lst.append("""play :{},sustain:({}),sustain_level:({})\nsample :bass_hit_c\nsleep({})""".format(i[0], i[1], i[1], i[1]))
sonic_note_lst = '\n'.join(sonic_note_lst)

f = open("SonicPiCode/cube_notefile.txt", "w")
f.write(sonic_note_lst)

sonic_chord_lst = []
for i in zip(chords, rythyms):
    chord_major = i[0].split()
    sonic_chord_lst.append("""play chord(:{}, :{});sample :bass_hit_c\nsleep({})""".format(chord_major[0], chord_major[1], i[1]))
sonic_chord_lst = '\n'.join(sonic_chord_lst)

x = open("SonicPiCode/cube_chordfile.txt", "w")
x.write(sonic_chord_lst)
x.close()

<h2 align="center">
Basak-Miranda Algorithm
</h2>

The previous examples for the Quantum Random Walk only work in certain spaces given certain conditions. Cubes and Tetrahedrons have a predefined structure that makes the dice representation simple. However, if we try to extend this to other shapes such as a dodecahedron, the dice complexity will increase significantly. To generalize a Quantum Random Walk over any set of sequencing rules, the Basak-Miranda Algorithm makes use of Grover's Search Algorithm to not only construct an equivalent quantum circuit, but also provide a quadratic time speedup. 

## Grover's Search Algorithm

Grover's Search Algorithm is used to provide a quadratic speedup for the following problem statement: Given f: {0,...,N-1} -> {0,1} such that f(x)=1 for exactly one x, find x.

In other words, we use this algorithm to return a target state given a large unstructured list. The first step is called Phase Inversion, followed by Inversion about the Mean. The process can be shown in the following figure:

<figure align = "center">
  <img src="images/Picture10.png" alt="Grovers Algorithm">
  <figcaption>Figure #8</figcaption>
  <figcaption align="center">Source: Intrigano (https://www.youtube.com/watch?v=9WAxOlYBE3g&ab_channel=intrigano)</figcaption>
</figure>

The first diagram depicts a superposition of all states. Next, the target state has it's amplitude flipped. Finally, all values are inverted about the mean value. Thus, the target value has the highest amplitude, and after enough repetitions of this process, it is very likely that it will be the measured outcome. 

Grover's Search Algorithm can be applied in the case of algorithmic music generation via sequencing rules. Firstly, we create a Markov Chain based on our chosen sequencing rules. After assigning each state a vertex and maintaining relevant dictionaries, we can choose the target states that have been predefined by our Markov Chain based on the current vertex we are present on. The output of Grover's Search Algorithm will be only the states that have been defined in our sequencing rules. Repeating this process with a new current vertex will give us a musical composition for any kind of sequencing rules. Not just that, but it provides a quadratic time speedup, as well as naturally assigning equal amplitudes, and thus probabilities, to our target states. 

## Creating Complex Sequencing Rules 

These sequencing rules are defined in the paper, "Quantum Computer Music: Foundations and Initial Experiments" by Miranda and Basak. The rules themselves are maintained in a dictionary, as well as the note to vertex and vertex to note correspondence. The vertex to note dictionary is also maintained in decimal form for convenience.


In [11]:
notes = ["E", "F", "G", "C#", "F#", "D#", "G#", "D", "B", "C", "A", "A#"]
sequencing_rules = {i:[] for i in notes}

sequencing_rules["E"] = ["F", "D#"]
sequencing_rules["D#"] = ["F", "C#", "F#", "G#"]
sequencing_rules["C#"] = ["G", "F#", "D#"]
sequencing_rules["G"] = ["F", "C#", "D"]
sequencing_rules["D"] = ["F", "G", "G#", "B"]
sequencing_rules["F"] = ["E", "G", "D", "C"]
sequencing_rules["C"] = ["F", "F#", "B", "A"]
sequencing_rules["F#"] = ["C#", "D#", "C", "A"]
sequencing_rules["A"] = ["F#", "G#", "C", "A#"]
sequencing_rules["G#"] = ["D#", "D", "B", "A"]
sequencing_rules["B"] = ["G#", "D", "C", "A#"]
sequencing_rules["A#"] = ["B", "A"]

note_to_vertex = {}

note_to_vertex["E"] = "0000"
note_to_vertex["F"] = "0001"
note_to_vertex["G"] = "0010"
note_to_vertex["C#"] = "0011"
note_to_vertex["F#"] = "0100"
note_to_vertex["D#"] = "0101"
note_to_vertex["G#"] = "0110"
note_to_vertex["D"] = "0111"
note_to_vertex["B"] = "1000"
note_to_vertex["C"] = "1001"
note_to_vertex["A"] = "1010"
note_to_vertex["A#"] = "1011"

vertex_to_note = {}

vertex_to_note["0000"] = "E"
vertex_to_note["0001"] = "F"
vertex_to_note["0010"] = "G"
vertex_to_note["0011"] = "C#"
vertex_to_note["0100"] = "F#"
vertex_to_note["0101"] = "D#"
vertex_to_note["0110"] = "G#"
vertex_to_note["0111"] = "D"
vertex_to_note["1000"] = "B"
vertex_to_note["1001"] = "C"
vertex_to_note["1010"] = "A"
vertex_to_note["1011"] = "A#"
vertex_to_note["1100"] = "E"
vertex_to_note["1101"] = "F"
vertex_to_note["1110"] = "G"
vertex_to_note["1111"] = "C#"

vertices = {}

vertices[0] = "E"
vertices[1] = "F"
vertices[2] = "G"
vertices[3] = "C#"
vertices[4] = "F#"
vertices[5] = "D#"
vertices[6] = "G#"
vertices[7] = "D"
vertices[8] = "B"
vertices[9] = "C"
vertices[10] = "A"
vertices[11] = "A#"

## Creating a Markov Chain

In [12]:
def markov():

    markov = [[0 for i in range(len(sequencing_rules.keys()))] for i in range(len(sequencing_rules.keys()))]

    for i, j in vertices.items():
        
        for x in range(len(markov[i])):

            if vertices[x] in sequencing_rules[vertices[i]]:

                markov[i][x] = round(1/len(sequencing_rules[vertices[i]]), 2)

    return markov    

## Building the Circuit for the Basak-Miranda Algorithm

The first step of Grover's Search Algorithm, Phase Inversion, can be implemented with the use of X Gates and MCX Gates. After initializing the circuit in a superposition of states, wherever there is a 1, an X gate is applied. We have an extra qubit than measured qubits, which is initialized in the $|->$ state, and the target qubit of the MCX gate is on this extra qubit. X gates are reapplied in the same way, as well as Hadamard gates. What this circuit will do is flip the target states. To do it for multiple target states, we repeat it as many times as target states.

Next, we create the circuit to implement Inversion about the Mean. The source of this circuit was taken from Qiskit<sup>[1](#f1)</sup>, and the idea is to implement a Diffuser that can be created by Hadamard and X gates, followed by a Hadamard, MCT, and Hadamard gate on the second last qubit, finally reapplying the X and Hadamard gates.

<b id="f1">1</b> Source: https://qiskit.org/textbook/ch-algorithms/grover.html.

In [13]:
def basak_miranda_circuit(input):

    qc = QuantumCircuit(5,4)

    qc.x(4)
    for qubit in range(5):
        qc.h(qubit)

    qc.barrier()
    for i in input:

        flip = []

        for j in range(len(i)):
            if i[j] == '1':
                flip.append(j)

        for j in flip:
            qc.x(j)

        if len(flip) >= 1:
            qc.mcx(flip, 4)
        
        for j in flip:
            qc.x(j)

        qc.barrier()

    for qubit in range(4):
        qc.h(qubit)
    

    qc.barrier()
    for qubit in range(4):
        qc.h(qubit)
    
    for qubit in range(4):
        qc.x(qubit)

    qc.barrier()
    qc.h(3)
    qc.mct([0,1,2], 3)
    qc.h(3)
    qc.barrier()

    for qubit in range(4):
        qc.x(qubit)

    for qubit in range(4):
        qc.h(qubit)
    qc.barrier()

    return qc

## Implementing the Basak-Miranda Algorithm

Starting off with the vertex '0000', we do a Quantum Random Walk 10 times, passing the target states as an argument into our circuit implementation function. We find these target states by referring to positive values in our markov chain for each current vertex. At the end of these iterations we convert the vertices to their respective notes and rythyms, as represnted in the dictionaries given above.

In [14]:
def quantum_walk():

    start = "0000"
    vertex_lst = []
    note_lst = []
    markov_chain = markov()

    for i in range(10):

        target_states = []

        for j in range(len(markov_chain[notes.index(vertex_to_note[start])])):
            if markov_chain[notes.index(vertex_to_note[start])][j] > 0:
                target_states.append(note_to_vertex[notes[j]])
                
        temp = basak_miranda_circuit(target_states)
        
        temp.measure(range(4),range(4))

        job = execute(temp,Aer.get_backend('qasm_simulator'),shots=50000)
        counts = list(job.result().get_counts(temp).items())

        counts.sort(key=lambda x: x[1])

        i = -1
        while counts[i][0] in ['1111', '1110', '1101', '1100']:
            i+=1
        start = counts[i][0]
        vertex_lst.append(start)

    for i in vertex_lst:
        if i in vertex_to_note:
            note_lst.append(vertex_to_note[i])
        
    print(vertex_lst)
    
    return note_lst

notes = quantum_walk()

['1000', '0001', '0000', '1000', '0010', '1000', '0010', '1000', '0001', '0000']


## Sonic Pi code created Using Basak-Miranda Algorithm

We save the code usable with the application, SonicPi, into text files. The sleep function is added in place for the rhythym, and some sustain and bass is also added. 

In [15]:
sonic_note_lst = []
for i in zip(notes, rythyms):
    sonic_note_lst.append("""play :{},sustain:({}),sustain_level:({})\nsample :bass_hit_c\nsleep({})""".format(i[0], i[1], i[1], i[1]))
sonic_note_lst = '\n'.join(sonic_note_lst)

f = open("SonicPiCode/basak_miranda_notefile.txt", "w")
f.write(sonic_note_lst)
f.close()

<h2 align="center">
Conclusion
</h2>

To conclude this paper, I will firs summarize the main results and takeaways from this project. The main goal throughout all this was to generate a musical composition. This comprises of, but is not limited to, notes and rhythyms. As an extra result, musical composition containing purely chords was also created for a Quantum Random Walk over a Cube. Moving on to theh more complex sequencing rules, Grover's Algorithm was implemented to generate a musical composition. However, there were issues in implementation, and the resulting circuit produces incorrect transitions between states, based on predefined sequencing rules. 
One of the main results from this project, however, is that a quantum circuit was able to be generated for a Tetrahedron. The cited and followed research paper creates a quantum dice for a cube, and it is possible to create quantum dice for other figures. Further research could be conducted on generalising the idea of a quantum die over platonic solids (tetrahedron, cube, etc.). 
Using SonicPi for music creation allows for much more creativity, especicially over the traditional method of 'beep-ing' the frequency corresponding to a note. SonicPi allows for chords to be played without mentioning the corresponding notes. It allows simultaneous playback, and interesting elements such as sustain and bass. 

Overall, Quantum Computer Music is a new and exciting way of using a well-known algorithm to allow convenience and quadratic time speedup in something one would not expect a Quantum Computer to provide specific use. Miranda and Basak have created an algorithm and exposed the world to a new and creative use of Quantum Circuits. The use of Quantum Dice and Quantum Random Walks could further have use in many different, and possibly unrelated, fields.

<h2 align="center">
References
</h2>

1. Quantum Computer Music: Foundations and Initial Experiments (Miranda, Basak)
2. Intrigano (https://www.youtube.com/watch?v=9WAxOlYBE3g&ab_channel=intrigano)
3. Grover's Search Algorithm- Qiskit (https://qiskit.org/textbook/ch-algorithms/grover.html)
